In [1]:
import numpy as np
import pyci
import psi4

In [2]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('aug-cc-pvqz', 'He.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('mo_scf_info.npz')['eps_a']
Ca = np.load('mo_scf_info.npz')['Ca'] 
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints)
del ao_erints

Size of the ERI tensor will be 0.04 Gb

Ground state SCF Energy : -2.86152200 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -2.86152200 



In [3]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 46  
Molecular orbitals: 46  
Spin Orbitals     : 92  
Alpha Orbitals    : 1  
Beta Orbitals     : 1  
Occupied Orbitals : 1  
Virtual Orbitals  : 45  



In [4]:
scf_energy = aoint.scf_energy
psi4_cisd = psi4.energy('CISD')

In [5]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
singles = True
full_cis = True
doubles = True
doubles_iiaa = True
doubles_iiab = True
doubles_ijaa = True
doubles_ijab_A = True
doubles_ijab_B = True
options = [singles, full_cis, doubles,
           doubles_iiaa, doubles_iiab,doubles_ijaa,
           doubles_ijab_A, doubles_ijab_B]
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)


([1, 45, 45, 990, 0, 0, 0], 1081)

In [6]:
type(csfs)

list

In [7]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=2)

In [8]:
HCISD.shape

(1081, 1081)

In [9]:
vals, vecs = np.linalg.eigh(HCISD)

In [10]:
# Comparing pyci with psi4
print("pyci CISD E0: {e:16.16f}".format(e=vals[0]))
print("psi4 CISD E0: {e:16.16f}".format(e=psi4_cisd))
print("dE : {dE:1.2E}".format(dE=abs(psi4_cisd-vals[0])))


pyci CISD E0: -2.9025335994179482
psi4 CISD E0: -2.9025335994179469
dE : 1.33E-15
